In [2]:
from tabulate import tabulate
from termcolor import colored
import mysql.connector

connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='root@123',
    database='lms_db',
)
cursor = connection.cursor()

def issue_book():
    
    print("Enter book issuance details:")
    try:
        member_id = int(input("Enter the member ID: "))
        book_id = int(input("Enter the book ID: "))
        
        # Check if the member exists
        sel_member = "SELECT * FROM member WHERE member_id = %s"
        cursor.execute(sel_member, (member_id,))
        member_record = cursor.fetchone()
        
        # Check if the book exists and has available copies
        sel_book = "SELECT * FROM book WHERE book_id = %s AND copies > 0"
        cursor.execute(sel_book, (book_id,))
        book_record = cursor.fetchone()
        
        if member_record and book_record:
            issue_id = int(input("Enter an issue ID: "))
            issue_date = input("Enter a date in YYYY-MM-DD format: ")
            copie = int(input("Enter the number of copies to issue: "))
            
            if copie > 0 and copie <= book_record[4]:  # Assuming rem_copies is at index 5
                # Insert into issue table
                ins_issue = 'INSERT INTO issue (issue_id, issue_date, member_id, book_id, copie) VALUES (%s, %s, %s, %s, %s)'
                ins_values_issue = (issue_id, issue_date, member_id, book_id, copie)
                cursor.execute(ins_issue, ins_values_issue)
                
                # Update rem_copies in the book table
                update_copies = "UPDATE book SET rem_copie = rem_copie - %s WHERE book_id = %s"
                cursor.execute(update_copies, (copie, book_id))
                
                connection.commit()
                print("Book issuance successful!")
            else:
                print("Invalid number of copies or insufficient available copies.")
        else:
            print('No such member exists or no available copies of the book')

    except mysql.connector.Error as error:
        print(f"MySQL error: {error}")


def return_book():
    print("Enter book return details:")
    try:
        issue_id = int(input("Enter the issue ID: "))
        
        sel_issue = "SELECT * FROM issue WHERE issue_id = %s"
        cursor.execute(sel_issue, (issue_id,))
        issue_record = cursor.fetchone()
        
        if issue_record:
            print("Issue Record:", issue_record)
    # Accessing specific indices
            copies= issue_record[4]  # Assuming copies are at index 5
            book_id = issue_record[3]  # Assuming book_id is at index 4
            print("Copies Returned:", copies)
            print("Book ID:", book_id)
            return_id = int(input("Enter an return ID: "))
            return_date = input("Enter a date in YYYY-MM-DD format: ")
            copie = int(input("Enter the number of copies to issue: "))
            member_id = int(input("Enter the member ID: "))
            book_id = int(input("Enter the book ID: "))
            ins_return = 'INSERT INTO returns (return_id, return_date, member_id, book_id, copies) VALUES (%s, %s, %s, %s, %s)'
            ins_values_return = (return_id, return_date, member_id, book_id, copies)
            cursor.execute(ins_return, ins_values_return)
            del_issue = "DELETE FROM issue WHERE issue_id = %s"
            cursor.execute(del_issue, (issue_id,))
            
            # Update rem_copies in the book table
            update_copies = "UPDATE book SET rem_copie = rem_copie + %s WHERE book_id = %s"
            cursor.execute(update_copies, (copies, book_id))
            
            connection.commit()
            print("Book return successful!")
        else:
            print('No such issued book exists')

    except mysql.connector.Error as error:
        print(f"MySQL error: {error}")



def status_book():
    print("enter")
    try:
        query ="select book_id,b_name,rem_copie,m_name from book inner join member on book.book_id=member.member_id"
        cursor.execute(query)
        results = cursor.fetchall()
        if results:
            headers = [i[0] for i in cursor.description]
            colored_headers = [colored(header, 'blue') for header in headers]
            print(tabulate(results, headers=colored_headers, tablefmt="pretty"))
            print('Book Search successfully')
    except mysql.connector.Error as error:
        print(f"mysql error: {error}")
#status_book()